In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import seaborn as sbn


dff=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

dff

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [2]:
for df in [dff,df_test]:
#   df["Cabin"]= df["Cabin"].str.split("/", n = 1, expand = True)[0]
    df.drop('Cabin',axis=1,inplace=True)
    df

In [3]:
for df in [dff,df_test]:
    df.drop(['Name','PassengerId'],axis=1,inplace=True)
    
    
    
    for i in ['CryoSleep','VIP','Transported']:
        try:
            df[i]=pd.get_dummies(df[i])[1]
        except:
            continue

    
    for i in ['HomePlanet','Destination']:
        df[i]=df[i].astype('category').cat.codes
    df

In [4]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
IMPUTE = True

In [5]:
def analyze_column(input_series: pd.Series) -> str:
    if pd.api.types.is_numeric_dtype(input_series):
        return 'numeric'
    else:
        return 'categorical'
    


class LGBMImputer:
    '''
    Regression imputer using LightGBM
    '''
    def __init__(self, cat_features=[], n_iter=15000, verbose=False):
        self.n_iter = n_iter
        self.cat_features = cat_features
        self.verbose = verbose
        self.n_features = None
        self.feature_names = None
        self.feature_with_missing = None
        self.imputers = {}
        self.offsets = {}
        self.objectives = {} 
                                                                            # Özellikler-Parametreler belirleniyor...
        
    def fit_transform(self, X, y=None):
        output_X = X.copy()
        self.n_features = X.shape[1]

        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns.tolist()
        else:
            self.feature_names = [f'f{i}' for i in range(self.n_features)]
            X = pd.DataFrame(X, columns=self.feature_names)
        self.feature_with_missing = [col for col in self.feature_names if X[col].isnull().sum() > 0]

        for icol, col in enumerate(self.feature_with_missing):              #icol sıra numarası, col kolon
            if icol in self.cat_features:
                nuni = X[col].dropna().nunique()
                if nuni == 2:
                    params = {
                        'objective': 'binary'
                    }
                elif nuni > 2:
                    params = {
                        'objective': 'multiclass',
                        'num_class': nuni + 1
                    }
            else: # automatic analyze column
                if analyze_column(X[col]) == 'numeric':
                    params = {
                        'objective': 'regression'
                    }
                else:
                    nuni = X[col].dropna().nunique()
                    if nuni == 2:
                        params = {
                            'objective': 'binary'
                        }
                    elif nuni > 2:
                        params = {
                            'objective': 'multiclass',
                            'num_class': nuni + 1
                        }
                    else:
                        print(f'column {col} has only one unique value.')
                        continue

            params['verbosity'] = -1
            
            null_idx = X[col].isnull()
            x_train = X.loc[~null_idx].drop(col, axis=1)
            x_test = X.loc[null_idx].drop(col, axis=1)
            y_offset = X[col].min()
            y_train = X.loc[~null_idx, col].astype(int) - y_offset
            dtrain = lgb.Dataset(
                data=x_train,
                label=y_train
            )

            early_stopping_rounds = 50
            model = lgb.train(
                params, dtrain, valid_sets=[dtrain], 
                num_boost_round=self.n_iter,
                early_stopping_rounds=early_stopping_rounds,
                verbose_eval=0,
            )

            y_test = model.predict(x_test)
            if params['objective'] == 'multiclass':
                y_test = np.argmax(y_test, axis=1).astype(float)
            elif params['objective'] == 'binary':
                y_test = (y_test > 0.5).astype(float)
            y_test += y_offset
            output_X.loc[null_idx, col] = y_test
            if params['objective'] in ['multiclass', 'binary']:
                output_X[col] = output_X[col].astype(int)
            self.imputers[col] = model
            self.offsets[col] = y_offset
            self.objectives[col] = params['objective']
            if self.verbose:
                print(f'{col}:\t{self.objectives[col]}...iter{model.best_iteration}/{self.n_iter}')
        
        return output_X    

In [6]:
if IMPUTE:
    s=0
    for df in [dff,df_test]:
        feature_df = df
        
        feature_df = feature_df.replace(99999.0,np.nan)

        imputer = LGBMImputer(verbose=True)
        feature_df.loc[:, feature_df.columns[1:]] = imputer.fit_transform(feature_df[feature_df.columns[1:]])

        if s==0:
            train_df = feature_df.copy().reset_index(drop=True)
            s=1
        else:
            test_df = feature_df.copy().reset_index(drop=True)

c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Age:	regression...iter15000/15000


c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


RoomService:	regression...iter15000/15000


c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

ShoppingMall:	regression...iter15000/15000


c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Spa:	regression...iter15000/15000


c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\mehmu\Desktop\PyProjects\env\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


VRDeck:	regression...iter15000/15000


In [7]:
test_df.isna().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64

In [8]:
train_df.to_csv('trainimp.csv')
test_df.to_csv('testimp.csv')